In [1]:
%reset -f

from time import strftime

import numpy as np
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
%matplotlib inline

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
DATASET = "cat-dog-25000-GrayScale-(50, 50).npy"
TEST_DIR = "./cat-dog-12500-GrayScale-(50, 50)-test.npy"
LR = 0.001
ROW_SIZE = 50
COL_SIZE = 50
EPOCHS = 1
CLASSES = 2 #[ DOG, CAT ]
MODEL_NAME = f"cat-dog-{EPOCHS}-6"

In [3]:
def generateData():
    data = np.load(DATASET, allow_pickle=True)
    np.random.shuffle(data)
    
    X = []
    y = []
    
    def to_categorical(y):
        if y.upper() == 'CAT':
            return [0,1]
        if y.upper() == 'DOG':
             return [1,0]
    
    for img in data:
        X.append(img[0])
        y.append(to_categorical(img[1]))
    X = np.array(X).reshape(-1, ROW_SIZE, COL_SIZE, 1)
    return X, np.array(y)


In [4]:
X, y = generateData()
X = X/255.0

test_x = X[23000:]
test_y = y[23000:]

X = X[:23000]
y = y[:23000]

convnet = input_data(shape=[None, ROW_SIZE, COL_SIZE, 1], name='input')

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 64, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir=f"../logs/alexandre/{MODEL_NAME}-{strftime('%d-%m-%Y')}-{strftime('%H-%M-%S')}")

model.fit({'input': X}, {'targets': y}, n_epoch=EPOCHS, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.save(f"{MODEL_NAME}.model")

Training Step: 359  | total loss: 0.58551 | time: 8.014s
| Adam | epoch: 001 | loss: 0.58551 - acc: 0.6969 -- iter: 22976/23000
Training Step: 360  | total loss: 0.58874 | time: 9.132s
| Adam | epoch: 001 | loss: 0.58874 - acc: 0.6882 | val_loss: 0.56803 - val_acc: 0.7095 -- iter: 23000/23000
--
INFO:tensorflow:C:\Users\Skzt-Desk\PycharmProjects\faculdade\Cat-Dog\cat-dog-1-6.model is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
model = model.load(f"{MODEL_NAME}.model")

In [ ]:
fig = plt.figure()
num = 0

testing_data = np.load(TEST_DIR, allow_pickle=True)
with open("result.txt", 'w') as fp:
    for img_data in testing_data:
        img = img_data[0]
        orig = img
        img = img.reshape(-1, ROW_SIZE, COL_SIZE, 1)
        prediction = model.predict(img)[0]
        str_label = f"Numero: {np.argmax(prediction)} ({img_data[1]})"
        output = ["{0:.2f}%".format(i * 100) for i in prediction]
        fp.write(f"{str(output)}\n")
        
        if num < 12:
            print(["{0:.2f}%".format(i * 100) for i in prediction])
            y = fig.add_subplot(3, 4, num + 1)
            y.imshow(orig, cmap='gray')
            plt.title(str_label)
            y.axes.get_xaxis().set_visible(False)
            y.axes.get_yaxis().set_visible(False)
        num += 1
    plt.show()


